In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import math
import env

DB_URL = f'mysql+pymysql://{env.user}:{env.password}@{env.host}/tidy_data'


### 1.) Attendance Data

#### Read the data from the attendance table.

In [2]:
df = pd.read_sql('''SELECT * FROM attendance''', DB_URL)
df

,Unnamed: 0,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08
0,Sally,P,T,T,H,P,A,T,T
1,Jane,A,P,T,T,T,T,A,T
2,Billy,A,T,A,A,H,T,P,T
3,John,P,T,H,P,P,T,P,P


In [3]:
df.columns = ['student', '1/1', '1/2', '1/3', '1/4', '1/5', '1/6','1/7', '1/8']
df


,student,1/1,1/2,1/3,1/4,1/5,1/6,1/7,1/8
0,Sally,P,T,T,H,P,A,T,T
1,Jane,A,P,T,T,T,T,A,T
2,Billy,A,T,A,A,H,T,P,T
3,John,P,T,H,P,P,T,P,P


#### Calculate an attendance percentage for each student. One half day is worth 50% of a full day, and 10 tardies is equal to one absence.

In [4]:
# 
df = df.melt(id_vars = ['student'], var_name = 'date', value_name = 'attendance')

# Assign values to each variable
df['attendance'] = df.attendance.map({'P': 1,'A':0,'H': .5, 'T':.9})

In [5]:
df.groupby(by='student').mean()

,attendance
student,
Billy,0.5250
Jane,0.6875
John,0.9125
Sally,0.7625


### 2.) Coffee Levels

#### Read the coffee_levels table.

In [6]:
df = pd.read_sql('''SELECT * FROM coffee_levels''', DB_URL)

df.head()

,hour,coffee_carafe,coffee_amount
0,8,x,0.816164
1,9,x,0.451018
2,10,x,0.843279
3,11,x,0.335533
4,12,x,0.898291


#### Transform the data so that each carafe is in it's own column.

In [7]:
df = df.pivot(index = 'hour', columns = 'coffee_carafe').reset_index()

In [8]:
df.columns.name= ''
df

hour coffee_amount                    
coffee_carafe                  x         y         z
0                8      0.816164  0.189297  0.999264
1                9      0.451018  0.521502  0.915990
2               10      0.843279  0.023163  0.144928
3               11      0.335533  0.235529  0.311495
4               12      0.898291  0.017009  0.771947
5               13      0.310711  0.997464  0.398520
6               14      0.507288  0.058361  0.864464
7               15      0.215043  0.144644  0.436364
8               16      0.183891  0.544676  0.280621
9               17      0.391560  0.594126  0.436677

#### Is this the best shape for the data?

It depends on what kind of conclusion you are trying to come to but, with little perspective, it makes better sense to have the hour be the index.

### 3.) Cake Recipes

#### Read the cake_recipes table. This data set contains cake tastiness scores for combinations of different recipes, oven rack positions, and oven temperatures.

In [14]:
df = pd.read_sql('''SELECT * FROM cake_recipes''', DB_URL)

df

,recipe:position,225,250,275,300
0,a:bottom,61.738655,53.912627,74.414730,98.786784
1,a:top,51.709751,52.009735,68.576858,50.228470
2,b:bottom,57.095320,61.904369,61.196980,99.248541
3,b:top,82.455004,95.224151,98.594881,58.169349
4,c:bottom,96.470207,52.001358,92.893227,65.473084
5,c:top,71.306308,82.795477,92.098049,53.960273
6,d:bottom,52.799753,58.670419,51.747686,56.183110
7,d:top,96.873178,76.101363,59.571620,50.971626


In [15]:
# Split at the ':'
df[['recipe', 'rack']] = df['recipe:position'].str.split(':', expand = True)


In [16]:
# Drop unneeded column that was split
df = df.drop(columns=('recipe:position'))
df

,225,250,275,300,recipe,rack
0,61.738655,53.912627,74.414730,98.786784,a,bottom
1,51.709751,52.009735,68.576858,50.228470,a,top
2,57.095320,61.904369,61.196980,99.248541,b,bottom
3,82.455004,95.224151,98.594881,58.169349,b,top
4,96.470207,52.001358,92.893227,65.473084,c,bottom
5,71.306308,82.795477,92.098049,53.960273,c,top
6,52.799753,58.670419,51.747686,56.183110,d,bottom
7,96.873178,76.101363,59.571620,50.971626,d,top


In [29]:
cake_melt = df.melt(id_vars= ['recipe', 'rack'], var_name = 'temp', value_name = 'rating')
cake_melt.head()

,recipe,rack,temp,rating
0,a,bottom,225,61.738655
1,a,top,225,51.709751
2,b,bottom,225,57.095320
3,b,top,225,82.455004
4,c,bottom,225,96.470207


#### Which recipe, on average, is the best? recipe b

In [31]:
cake_melt.groupby('recipe').mean()


,rating
recipe,
a,63.922201
b,76.736074
c,75.874748
d,62.864844


#### Which oven temperature, on average, produces the best results? 275

In [35]:
cake_melt.groupby('temp').mean()

,rating
temp,
225,71.306022
250,66.577437
275,74.886754
300,66.627655


#### Which combination of recipe, rack position, and temperature gives the best result? recipe b, bottom rack, 300 degrees


In [41]:
cake_melt.sort_values('rating').tail(1)

,recipe,rack,temp,rating
26,b,bottom,300,99.248541
